As usual we start loading the packages that we will use in our notebook

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder 


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/dataset/Advanced-NLP-TF/"

_about.txt  ita.txt  reviews.csv  test.csv  train.csv


In [0]:
dataset = "/content/drive/My Drive/dataset/Advanced-NLP-TF/train.csv"
train_df = pd.read_csv(dataset)#here we have the dataset we extracted
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [0]:
print(train_df.shape)

(20800, 5)


A key step is to label encode the target variable from text to number


In [0]:
Y = train_df['language']
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
Y = tf.keras.utils.to_categorical(
    Y,
    num_classes=4 #equals to the number of languages
    
)

KeyError: ignored

As we mentioned in the slides we will perform the previous text processing steps except for stopword removal.

In [0]:
train_df['sentence_lower'] = train_df["sentence"].str.lower()
train_df['sentence_no_punctuation'] = train_df['sentence_lower'].str.replace('[^\w\s]','')
train_df['sentence_no_punctuation'] = train_df["sentence_no_punctuation"].fillna("fillna")

In [0]:
max_features=5000 #we set maximum number of words to 5000
maxlen=400 #we set maximum sequence length to 400

In [0]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features) #again tokenizer step

In [0]:
tok.fit_on_texts(list(train_df['sentence_no_punctuation'])) #fit to cleaned text

In [0]:
print(len(tok.word_index))
vocab_size = len(tok.word_index) + 1 
#this represents the number of words that we tokenize different from max_features but necessary for
#the definition of the dimension of the embedding space

51630


In [0]:
train_df = tok.texts_to_sequences(list(train_df['sentence_no_punctuation'])) #this is how we create sequences
train_df = tf.keras.preprocessing.sequence.pad_sequences(train_df, maxlen=maxlen) #let's execute pad step

In [0]:

from sklearn.model_selection import train_test_split #divide into train and test set

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_df, Y, test_size=0.1, random_state=42)

In [0]:
embedding_dim = 50 #this is the final dimension of the embedding space.


Let's write down the model

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(input_dim=vocab_size, #embedding input
                           output_dim=embedding_dim,#embedding output
                           input_length=maxlen), #maximum length of an input sequence
  tf.keras.layers.Flatten(), #flatten layer

  tf.keras.layers.Dense(4, activation=tf.nn.softmax) #ouput layer a Dense layer with 4 probabilities
  #we also define our final activation function which is the softmax function typical for multiclass
  #classifiction problems

])

In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', #we recommend this loss function you
              metrics=['accuracy'])



In [0]:
model.summary() #here we show the architecture 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 50)           2581550   
_________________________________________________________________
flatten_2 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 80004     
Total params: 2,661,554
Trainable params: 2,661,554
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(np.array(X_train), np.array(y_train), epochs=3) #let's fit the model

C:\Users\Admin\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
3269/3269 [==============================] - 3s 1ms/step - loss: 0.8189 - acc: 0.7473
Epoch 2/3
3269/3269 [==============================] - 3s 950us/step - loss: 0.0801 - acc: 0.9966
Epoch 3/3
3269/3269 [==============================] - 3s 942us/step - loss: 0.0268 - acc: 0.9976


Remember the train_test_split? now we use the test to evaluate our model

In [0]:
model.evaluate(np.array(X_test), np.array(y_test)) 

364/364 [==============================] - 0s 221us/step


[0.013595647582845701, 1.0]

LOOKS LIKE WE HAVE A PERFECT MODEL!! 
LET'S TAKE A LOOK AT THE CONFUSION MATRIX OF OUR EVALUATION SET!!

In [0]:
from sklearn.metrics import confusion_matrix #we import this package from sklearn and output it
predictions = model.predict(X_test) #here we make predictions
cm = confusion_matrix(predictions.argmax(axis=1), y_test.argmax(axis=1))#we generate the confusion matrix

In [0]:
cm #well this is really perfect!

array([[ 91,   0,   0,   0],
       [  0,  88,   0,   0],
       [  0,   0, 102,   0],
       [  0,   0,   0,  83]], dtype=int64)

Let's try brand new text

In [0]:
#these are the codes for each language in order to evaluate properly
print('english', encoder.transform(['english']))
print('french', encoder.transform(['french']))
print('italian', encoder.transform(['italian']))
print('spanish', encoder.transform(['spanish']))

english [0]
french [1]
italian [2]
spanish [3]


In this experiment we will predict the language of the same sentence in the different languages

In [0]:
#new_text = ["tensorflow is a great tool you can find a lot of tutorials from packt"]
#new_text = ["tensorflow est un excellent outil vous pouvez trouver beaucoup de tutoriels de packt"]
#new_text = ["tensorflow è un ottimo strumento puoi trovare molti tutorial di packt"]
new_text = ["tensorflow es una gran herramienta puedes encontrar muchos tutoriales de packt"]


In [0]:
test_text = tok.texts_to_sequences(new_text) #this is how we create sequences
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=maxlen) #let's execute pad step

In [0]:
np.set_printoptions(suppress=True)
predictions = model.predict(test_text)
print(predictions.argmax())
print(predictions) #spanish you can get confused with italian which makes sense since they are more similar languages

3
[[0.05204275 0.0830349  0.10003749 0.7648848 ]]


In [0]:
import wikipedia

Let's build a brand new data set with only spanish and let's see if we recognize it ...

In [0]:
new_wiki_text = []
wikipedia.set_lang('es')
for i in range(0, 5):
    print(i)
    random = wikipedia.random(1)
       
    try:
        new_wiki_text.append([wikipedia.page(random).summary])
    except wikipedia.exceptions.DisambiguationError as e:
        random = wikipedia.random(1)

0
1
2
3
4


In [0]:
new_wiki_text = pd.DataFrame(new_wiki_text)
new_wiki_text.columns = ['sentence']
new_wiki_text

,sentence
0,Pontefract es un pueblo del distrito de Wakefi...
1,Joan de Giorgio Vitelli i Simon (Alguer 1870 -...
2,Glitter es una banda sonora original y el octa...
3,Bajo el apelativo de Cocinas del Sureste Asiát...
4,Neolucanus diffusus es una especie de coleópte...


In [0]:
new_wiki_text['sentence_lower'] = new_wiki_text["sentence"].str.lower()
new_wiki_text['sentence_no_punctuation'] = new_wiki_text['sentence_lower'].str.replace('[^\w\s]','')
new_wiki_text['sentence_no_punctuation'] = new_wiki_text["sentence_no_punctuation"].fillna("fillna")

In [0]:
np.set_printoptions(suppress=True)
test_wiki_text = tok.texts_to_sequences(list(new_wiki_text['sentence_no_punctuation'] )) #this is how we create sequences
test_wiki_text = tf.keras.preprocessing.sequence.pad_sequences(test_wiki_text, maxlen=maxlen) #let's execute pad step

In [0]:
predictions = model.predict(test_wiki_text)
print(predictions)

[[0.00000093 0.00010869 0.00000099 0.9998894 ]
 [0.00038548 0.00102032 0.00080112 0.9977931 ]
 [0.0000383  0.00060026 0.00007764 0.99928385]
 [0.00000019 0.00000787 0.00000026 0.99999166]
 [0.00212952 0.01033464 0.00334356 0.98419225]]


In [0]:
print('english', encoder.transform(['english']))
print('french', encoder.transform(['french']))
print('italian', encoder.transform(['italian']))
print('spanish', encoder.transform(['spanish']))

english [0]
french [1]
italian [2]
spanish [3]


WE DID A GOOD JOB!!